In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn as s

import matplotlib.pyplot as plt
%matplotlib inline


import math

# Parte 1

In [ ]:
dfw = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv" ,sep=',',engine='python')
dfw.head()

In [ ]:
dfw.describe(include="all")

In [ ]:
plt.figure(figsize =(12, 8))
sns.countplot("quality", data=dfw )

In [ ]:
plt.figure()
dfw.hist(rwidth=0.9)
plt.tight_layout()

In [ ]:
dfw.info()


In [ ]:
plt.subplots(figsize=(16,16))
sns.heatmap(dfw.corr(), annot=True)

In [ ]:
sns.pairplot(dfw)

Aca se pueden ver las correlaciones entre las variables en forma de grafica.

# Parte 2

In [ ]:
X = dfw.drop('quality',axis=1)
y = dfw['quality']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=30)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression().fit(X_train,y_train)

In [ ]:
lr.coef_

In [ ]:
coeff = pd.DataFrame(lr.coef_, X_train.columns, columns=['Coefficient'])
coeff

In [ ]:
print('Pendiente:', lr.coef_)
print('Término independiente :',lr.intercept_)

In [ ]:
predic = lr.predict(X_test)

In [ ]:
predic

In [ ]:
dfp = pd.DataFrame({'Actual':y_test,'Predictions':predic})
dfp['Predictions']= round(dfp['Predictions'],2)
dfp

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, predic))

In [ ]:
print('R squared: ',lr.score(X_train, y_train))

# - R squared:  0.3532840109858999
# - Mean squared error: 0.41

# Parte3

In [ ]:
def convert_quali (quality):
    if quality <= 5:
        return 0
    else:
        return 1

Bueno = 1
Malo = 0

In [ ]:
dfw["quality_bin"] = dfw["quality"].apply(convert_quali)

In [ ]:
dfw.head()

In [ ]:
dfw.dtypes

# Logistic Regression

In [ ]:
Xlr = dfw.drop(["quality" ,"quality_bin"] , axis=1)
ylr = dfw['quality_bin']


In [ ]:
Xlr.head()

In [ ]:
ylr.head()

In [ ]:
Xlr_train,Xlr_test,ylr_train,ylr_test= train_test_split(Xlr,ylr,test_size=0.2,random_state=30)

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(max_iter=1000) 
LR.fit(Xlr_train,ylr_train) 
ylr_prediccion = LR.predict(Xlr_test)

In [ ]:
import seaborn as sns; sns.set()
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ylr_test, ylr_prediccion)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('Clase de test')
plt.ylabel('Clase de la predicción');

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

accu_score = accuracy_score(ylr_test, ylr_prediccion)
print (classification_report(ylr_test, ylr_prediccion))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
probas = LR.predict_proba(Xlr_test)
fpr, tpr, thresholds = roc_curve(ylr_test, probas[:, 1])

plt.plot([0,1],[0,1],'k--') 
plt.plot(fpr, tpr, label='NB') 
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('Curva ROC')
plt.show()

print(roc_auc_score(ylr_test, probas[:, 1]))

In [ ]:
print('R squared: ',LR.score(Xlr_train,ylr_train))

In [ ]:
print('Mean squared error: %.2f'
      % mean_squared_error(ylr_test, ylr_prediccion))

- Accuracy   0.76 
- R squared:  0.745113369820172
- Mean squared error: 0.24

# K-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV

# For checking Model Performance
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve

import warnings
warnings.simplefilter(action="ignore")

from sklearn.model_selection import StratifiedKFold

In [ ]:
cv_method = StratifiedKFold(n_splits=3)
param={'n_neighbors':(1,3,5,7),'weights': ('uniform','distance'),'metric':('euclidean','manhattan','chebyshev','minkowski','wminkowski','seuclidean','mahalanobis'),'p' :(1,2)}
clf = GridSearchCV(neigh, param,cv=cv_method,scoring="accuracy")
clf.fit(Xlr_train,ylr_train)

In [ ]:
clf.best_params_

In [ ]:
knn = KNeighborsClassifier(n_neighbors= 7, p= 1, weights='distance',metric= "manhattan")
knn.fit(Xlr_train,ylr_train)
predic= knn.predict(Xlr_test)
matrix = confusion_matrix(ylr_test,predic)
sns.heatmap(matrix.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('Clase de test')
plt.ylabel('Clase de la predicción');

In [ ]:
accu_score = accuracy_score(ylr_test, predic)
print (classification_report(ylr_test, predic))

In [ ]:
knn.score(Xlr_test,ylr_test)

In [ ]:
print('Mean squared error: %.2f'
      % mean_squared_error(ylr_test, predic))

In [ ]:
print('R squared: ',knn.score(Xlr_test,ylr_test))

- R squared:  0.740625

## Linear Regression
- R squared: 0.3532840109858999
- Mean squared error: 0.41

## Logistic Regression
- Accuracy   0.76 
- R squared:  0.745113369820172
- Mean squared error: 0.24


## K-NN
- R squared:  0.740625
- Accuracy   0.74
- Mean squared error: 0.26



Podemos ver que hay una gran diferencia entre los modelos de Logistic Regression y K-nn  en compraracion a la regresion lineal. Donde estos dos modelos predicen mucho mejor al modelo de regression lineal, donde se puede ver que los r2 son mas del doble. Sinembarogo esto se puede deber a que la regresion lineal a diferencia del resto de los modelos busca predecir el valor exacto de la calidad del vino mientras los otros si son buenos o malos